In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
from google.colab import files
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
url = 'https://drive.google.com/file/d/1kELozbBlpVFRq0chK1G6ALB6y8OMpgYS/view?usp=sharing'
url= 'https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Grupo
**André Guarnier De Mitri - 11395579** \\
**Rui Emanuel Lima Vieira - 11810182** \\
**Luís Antonio Arruda Soares - 12559486**  \\
**Maria Victória Brandão Barros - 12608692** \\


# Informações básicas sobre os dados

In [ ]:
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


## Tipos de dados

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


- Podemos observar 3 colunas categóricas e 4 colunas quantitativas.

## Procurando valores nulos na amostra

In [ ]:
df.isnull().sum()

Name           0
Author         0
User Rating    0
Reviews        0
Price          0
Year           0
Genre          0
dtype: int64

- Não há valores nulos na amostra

# Tratando valores duplicados

In [ ]:
coluna_categorica = list(df.select_dtypes(exclude=('int', 'float')).columns)
print(f'Dados categoricos: {", ".join(coluna_categorica)}.')
for col in coluna_categorica:
    if df[col].duplicated().any() == True:
        print (f'Coluna {col} contem dados duplicados.')
    else:
        print (f'Coluna {col} não contem dados duplicados.')

Dados categoricos: Name, Author, Genre.
Coluna Name contem dados duplicados.
Coluna Author contem dados duplicados.
Coluna Genre contem dados duplicados.


In [ ]:
df.Genre.unique()

array(['Non Fiction', 'Fiction'], dtype=object)

- Genero não precisa ser atualizado

Atualizando os nomes para que, por exemplo, "  J. K  " seja igual a "J. K."

In [ ]:
df.Author = df.Author.str.title().str.strip()
df.Name = df.Name.str.title().str.strip()

### Importando uma biblioteca para arrumar nomes incoerentes

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import process

"George R.R. Martin" precisa ser igual a "George R.R. Martin" \\
"J.K. Rowling" precisa ser igual a "J. K. Rowling"

In [ ]:
authors = df.Author.sort_values().unique()
matches_author_name = fuzzywuzzy.process.extract('George R.R. Martin', authors, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches_author_name

[('George R. R. Martin', 100),
 ('George R.R. Martin', 100),
 ('George Orwell', 53),
 ('Bill Martin Jr.', 52),
 ('Mark R. Levin', 48)]

In [ ]:
df = df.replace('George R. R. Martin', 'George R.R. Martin')
df = df.replace('J. K. Rowling', 'J.K. Rowling')

## Removendo livros repetidos

Há livros repetidos em que a única mudança é o ano que ele participou dos 50 best sellers.

In [ ]:
df.tail()

,Name,Author,User Rating,Reviews,Price,Year,Genre
545,Wrecking Ball (Diary Of A Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are A Badass: How To Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are A Badass: How To Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are A Badass: How To Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction
549,You Are A Badass: How To Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2019,Non Fiction


Vamos remover essas repetições, deixando apenas o último ano. Isso impedirá que esses dados duplicados atrapalhem nossa análise.

In [ ]:
df_com_repeticoes = df
df = df.drop_duplicates(subset='Name', keep='last')
df.tail()

,Name,Author,User Rating,Reviews,Price,Year,Genre
538,Winter Of The World: Book Two Of The Century T...,Ken Follett,4.5,10760,15,2012,Fiction
539,Women Food And God: An Unexpected Path To Almo...,Geneen Roth,4.2,1302,11,2010,Non Fiction
544,Wonder,R. J. Palacio,4.8,21625,9,2017,Fiction
545,Wrecking Ball (Diary Of A Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
549,You Are A Badass: How To Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2019,Non Fiction


# Visualização dos dados

Contagem de livros de cada gênero

In [ ]:
df['Genre'].value_counts()

Non Fiction    190
Fiction        160
Name: Genre, dtype: int64

## Visualizando a contagem de livros por gênero


In [ ]:
temp_df = df_com_repeticoes['Genre'].value_counts().reset_index()
fig = px.pie(temp_df, values='Genre', 
             hover_data=['Genre'], labels={'Genre':'Numero de livros', 'index':'Genero'},
             names='index',
             title="Há mais livros de Não Ficção do que de Ficção", 
             template="plotly_dark")
fig.show()

Há mais livros de Não Ficção do que de Ficção

## Quantidade por gênero ao longo dos anos

In [ ]:
d1 = df_com_repeticoes[df_com_repeticoes["Genre"] == "Fiction"]
d2 = df_com_repeticoes[df_com_repeticoes["Genre"] == "Non Fiction"]

col ="Year"

vc1 = d1[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Scatter(
                    x=vc1[col], 
                    y=vc1["count"], 
                    name="Fiction", 
                    marker=dict(color = 'red',
                             line=dict(color='white',width=1.5)))

trace2 = go.Scatter(
                    x=vc2[col], 
                    y=vc2["count"], 
                    name="Non Fiction", 
                    marker= dict(color = 'blue',
                              line=dict(color='white',width=1.5)))
layout = go.Layout(hovermode= 'closest', title = 'Número de Bestsellers por Gênero ao longo dos anos' , xaxis = dict(title = 'Ano'), yaxis = dict(title = 'Número de livros'), template= "plotly_dark")
fig = go.Figure(data = [trace1, trace2], layout=layout)
fig.show()

Os livros de não ficção sempre predominaram, exceto no ano de 2014

## Valor da nota por número de reviews e preço ao longo dos anos

Criando um data frame para valores medios

In [ ]:
df_medias = df.groupby(['Year', 'Genre']).agg(media_review=('Reviews', 'mean'), media_user_rating = ('User Rating', 'mean'), media_preco = ('Price', 'mean')).reset_index()
df_medias.head()

,Year,Genre,media_review,media_user_rating,media_preco
0,2009,Fiction,5402.300000,4.580000,17.550000
1,2009,Non Fiction,1875.388889,4.622222,13.555556
2,2010,Fiction,5752.357143,4.592857,9.571429
3,2010,Non Fiction,1983.105263,4.505263,13.526316
4,2011,Fiction,6610.000000,4.593333,11.600000


Aqui podemos ver como a Média do número de Reviews variou de acordo com a Média da Nota dada ao decorrer dos anos. \\
Quanto maior o tamanho do marcador, maior o Preço Médio.

In [ ]:
px.scatter(df_medias, x="media_review", y="media_user_rating", animation_frame="Year", animation_group="Genre",
           size="media_preco", color="Genre", hover_name="Genre",
           labels={
                     "Year": "Ano",
                     "Genre": "Gênero",
                     "media_review": "Média anual Reviews",
                     "media_user_rating": "Média anual Notas",
                     "media_preco": "Média anual Preço"
                 },
           template= "plotly_dark", size_max=20, range_x= [1500, 25000], range_y= [4.2, 4.9])

- Acima, podemos observar que os livros de Ficção sempre receberam mais avaliações do que os de Não Ficção

## Plot 3D Media preço por Media de Reviews por Média de Nota

In [ ]:
fig = px.scatter_3d(df_medias, x='media_review', y='media_user_rating', z='media_preco',
                    color='Genre', symbol='Genre', template= "plotly_dark")
fig.update_layout(width=700, height=700)

## Boxplot das Notas dadas pelos usuarios

In [ ]:
fig = px.box(df, x="User Rating", y="Genre", color= "Genre", template="plotly_dark")
fig.show()

- As médias das notas dos dois apresentam valores distoantes
- Intervalo com a 50% das observações é maior nos livros de ficção. 
- Livros de Ficção possuem mais outliers do que os de Não Ficção.

# Testes de Hipoteses


## Teste de Shapiro Wilk para descobrir se as notas seguem a distribuição normal:
$H_0:$ Notas apresentam distribuição normal contra

$H_1:$ Notas NÃO apresentam distribuição normal

In [ ]:
# Alfa
alpha=0.05

# Teste de Shapiro Wilk
stat, pval = shapiro(df['User Rating'])

print('Teste Shapiro-Wilk:', f'{stat:.3f}')
print('P-Valor:', f'{pval:.20f}')
    
# Condição que aceita ou rejeita H0
if pval < alpha:
    print('Rejeita H0, portanto as notas NÃO apresentam distribuição normal.')
else:
    print('Aceita H0, portanto as notas apresentam distribuição normal.')

Teste Shapiro-Wilk: 0.877
P-Valor: 0.00000000000000045166
Rejeita H0, portanto as notas NÃO apresentam distribuição normal.


#### Notas não se aproximam da distribuição normal.

## Teste de Hipotese para comparar a distribuição das notas dos livros de ficção e não ficção:
Sendo $X_1, \cdots, X_n$ as notas dos livros de ficção $\mu_1$ sua media, e $Y_1, \cdots, Y_n$ a nota dos livros de não ficção, com média $\mu_2$, temos:

$H_0:$ NÃO EXISTEM diferenças significativas entre as distribuições dos dois grupos

$H_1:$ EXISTEM diferenças significativas entre as distribuições dos dois grupos

Separando os grupos de ficção e não ficção e suas respectivas notas:

In [ ]:
non_fiction = df[df['Genre'] == 'Non Fiction']['User Rating']
fiction = df[df['Genre'] == 'Fiction']['User Rating']

### Teste não paramétricos de Mann-Whitney

In [ ]:
# Alfa
alpha=0.05

# Teste U de Mann-Whitney
stat, pval = mannwhitneyu(non_fiction, fiction)

print('Estatística do teste Mann-Whitney:', f'{stat:.3f}')
print('P-Valor:', f'{pval:.20f}')

# Condição que aceita ou rejeita H0
if pval < alpha:
    print('Rejeita H0, portanto EXISTEM diferenças significativas entre as distribuições dos dois grupos.')
else:
    print('Aceita H0, portanto NÃO existem diferenças significativas entre as distribuições dos dois grupos.')

Estatística do teste Mann-Whitney: 13013.000
P-Valor: 0.00931903468696572077
Rejeita H0, portanto EXISTEM diferenças significativas entre as distribuições dos dois grupos.


#### Rejeitamos H0, portanto EXISTEM diferenças significativas entre as distribuições dos dois grupos.

### Mediana dos dois tipos de livros

In [ ]:
print("Mediana dos livros de ficcao",  f'{non_fiction.median():.2f}', '\nMediana dos livros de Não Ficção', f'{fiction.median():.2f}')

Mediana dos livros de ficcao 4.60 
Mediana dos livros de Não Ficção 4.70


### Plotando a Distribuição de Densidade das Notas por gênero de livro

In [ ]:
# Cria displot com curve_type = 'normal'
fig = ff.create_distplot([non_fiction, fiction], 
                         ['Non_fiction', 'Fiction'], 
                         colors=['red', 'teal',],
                         bin_size=.1, 
                         show_rug=False)
fig.update_layout(title_text='Distribuição de densidade das Notas por gênero de livro', 
                     template= "plotly_dark")
fig.show()

### Normalizando os dados com a transformação Box-Cox


In [ ]:
BoxCox, lam = stats.boxcox(df['User Rating'])

In [ ]:
# Alfa
alpha=0.05

# Teste de Shapiro Wilk
stat, pval = shapiro(BoxCox)

print('Estatística Shapiro-Wilk:', f'{stat:.3f}')
print('P-Valor:', f'{pval:.20f}')
    
# Condição que aceita ou rejeita H0
if pval < alpha:
    print('Rejeita H0, portanto as Notas NÃO se aproximam da distribuição normal.')
else:
    print('Aceita H0, portanto as Notas se aproximam da distribuição normal.')

Estatística Shapiro-Wilk: 0.957
P-Valor: 0.00000001283832862242
Rejeita H0, portanto as Notas NÃO se aproximam da distribuição normal.


#### Não foi possível normalizar os dados com a transformação Box-Cox

# Conclusão

  Segundo o Teste de Shapiro Wilk, as notas dos livros não se aproximam da distribuição normal e não puderam ser normalizadas pela transformação Box-Cox \\
  E conforme o teste Mann-Whitney, podemos dizer com 95% de confiança que existem diferença significativa entre as distribuição das notas dos livros de Ficção e Não Ficção, já que rejeitamos a hipótese $H_0$.
